In [ ]:
import os
import subprocess
import wave
import speech_recognition as sr
from pydub import AudioSegment
from pydub.silence import split_on_silence
from datetime import datetime

# Для пунктуации
from deepmultilingualpunctuation import PunctuationModel

# ==== НАСТРОЙКИ ====
FFMPEG_PATH = r"C:/Users/olga/ffmpeg/bin/ffmpeg.exe"
OUTPUT_DIR = "chunks"
RESULT_FILE = "full_text.txt"
LANGUAGE = "en-EN"

# ==== ПОИСК АУДИО ====
AUDIO_EXTENSIONS = (".wav", ".mp3", ".m4a", ".flac", ".ogg")
audio_files = [
    f for f in os.listdir()
    if f.lower().endswith(AUDIO_EXTENSIONS) and f != "converted.wav"
]

if not audio_files:
    raise FileNotFoundError(f"В папке {os.getcwd()} не найдено аудиофайлов {AUDIO_EXTENSIONS}")

INPUT_FILE = audio_files[0]
print(f"[1/6] Найден файл: {INPUT_FILE}")

# ==== ПРОВЕРКА ФОРМАТА ====
needs_conversion = True
if INPUT_FILE.lower().endswith(".wav"):
    with wave.open(INPUT_FILE, "rb") as wav_file:
        channels = wav_file.getnchannels()
        framerate = wav_file.getframerate()
        sampwidth = wav_file.getsampwidth()
        print(f"  → Формат: {channels} канал(ов), {framerate} Hz, {sampwidth*8}-бит")
        if channels == 1 and framerate == 16000 and sampwidth == 2:
            needs_conversion = False

# ==== КОНВЕРТАЦИЯ ====
if needs_conversion:
    converted_file = "converted.wav"
    print(f"[2/6] Конвертация {INPUT_FILE} → {converted_file}")
    subprocess.run([
        FFMPEG_PATH, "-y",
        "-i", INPUT_FILE,
        "-ac", "1",
        "-ar", "16000",
        "-c:a", "pcm_s16le",
        converted_file
    ], check=True)
else:
    converted_file = INPUT_FILE
    print("[2/6] Конвертация не требуется — формат уже подходит")

# ==== РАЗБИВКА ====
print("[3/6] Разбивка на чанки...")
audio = AudioSegment.from_wav(converted_file)
chunks = split_on_silence(
    audio,
    min_silence_len=500,
    silence_thresh=audio.dBFS - 14,
    keep_silence=300
)

os.makedirs(OUTPUT_DIR, exist_ok=True)
chunk_files = []
for i, chunk in enumerate(chunks):
    chunk_path = os.path.join(OUTPUT_DIR, f"chunk_{i}.wav")
    chunk.export(chunk_path, format="wav")
    chunk_files.append(chunk_path)

print(f" → {len(chunk_files)} чанков сохранено")

# ==== РАСПОЗНАВАНИЕ ====
print("[4/6] Распознавание...")
recognizer = sr.Recognizer()
raw_text_parts = []

for idx, chunk_path in enumerate(chunk_files, 1):
    try:
        with sr.AudioFile(chunk_path) as source:
            recognizer.adjust_for_ambient_noise(source, duration=0.3)
            audio_data = recognizer.record(source)
            text = recognizer.recognize_google(audio_data, language=LANGUAGE)
            raw_text_parts.append(text)
            print(f"[{idx}/{len(chunk_files)}] ✔ {chunk_path}")

    except sr.UnknownValueError:
        raw_text_parts.append("")
        print(f"[{idx}/{len(chunk_files)}] ✖ Не распознан")
    except Exception as e:
        raw_text_parts.append("")
        print(f"[{idx}/{len(chunk_files)}] ⚠ Ошибка: {e}")

raw_text = " ".join(raw_text_parts)

# ==== ПУНКТУАЦИЯ ====
# print("[5/6] Расстановка пунктуации...")
# model = PunctuationModel()
# punctuated_text = model.restore_punctuation(raw_text)

# ==== ПУНКТУАЦИЯ ====
print("[5/6] Расстановка пунктуации...")
try:
    model = PunctuationModel()
    
    # Разбиваем текст на осмысленные части
    sentences = [s.strip() for s in raw_text.split('.') if s.strip()]
    punctuated_text = ""
    
    for sent in sentences:
        try:
            # Убрали параметр language, так как он не поддерживается
            processed = model.restore_punctuation(sent)
            
            # Добавляем точку, если в конце предложения её нет
            if not processed.endswith(('.', '!', '?')):
                processed += '.'
                
            punctuated_text += processed + " "
            
        except Exception as e:
            print(f"⚠ Ошибка при обработке предложения: {e}")
            punctuated_text += sent + ". "
    
    # Постобработка
    punctuated_text = punctuated_text.replace(" .", ".").replace(" ,", ",")
    punctuated_text = punctuated_text.strip()
    
except Exception as e:
    print(f"⚠ Критическая ошибка пунктуации: {e}")
    punctuated_text = raw_text  # Возвращаем исходный текст в случае ошибки

# ==== СОХРАНЕНИЕ ====
with open(RESULT_FILE, "w", encoding="utf-8") as f:
    f.write(punctuated_text)

print("[6/6] Готово!")
print(f"Текст сохранён в {RESULT_FILE} ({datetime.now().strftime('%H:%M:%S')})")
